# Neuromatriarchy

This notebook is based on [Deep Dream](https://github.com/google/deepdream/blob/master/dream.ipynb) notebook and is used for developer's convinience.

It'll be interesting to see what imagery people are able to generate using the described technique. If you post images to Google+, Facebook, or Twitter, be sure to tag them with **#deepdream** so other researchers can check them out too.

##Dependencies
The Neuromatriarchy progect inherrits dependencies from Deep Dream:
* Standard Python scientific stack: [NumPy](http://www.numpy.org/), [SciPy](http://www.scipy.org/), [PIL](http://www.pythonware.com/products/pil/), [IPython](http://ipython.org/). Those libraries can also be installed as a part of one of the scientific packages for Python, such as [Anaconda](http://continuum.io/downloads) or [Canopy](https://store.enthought.com/).
* [Caffe](http://caffe.berkeleyvision.org/) deep learning framework ([installation instructions](http://caffe.berkeleyvision.org/installation.html)).
* Google [protobuf](https://developers.google.com/protocol-buffers/) library that is used for Caffe model manipulation.

In [ ]:
# imports and basic notebook setup
from glob import glob
import numpy as np
import PIL.Image
from google.protobuf import text_format

import caffe

# If your GPU supports CUDA and Caffe was built with CUDA support,
# uncomment the following to run Caffe operations on the GPU.
# caffe.set_mode_gpu()
# caffe.set_device(0) # select GPU device if multiple devices exist

from np_array_utils import *
from dream_utils import *
from obsession_utils import *

# Original emotion recognition model
model_path = '/Users/sitin/Documents/Workspace/caffe/models/VGG_S_rgb/' # substitute your path here
    
emotions = Dreamer(
    net_fn=model_path + 'deploy.txt',
    param_fn=model_path + 'EmotiW_VGG_S.caffemodel',
    end_level='pool5'
)

##  Producing dreams

Here we will start from the death mask of delightful [Louise of Mecklenburg-Strelitz](https://en.wikipedia.org/wiki/Louise_of_Mecklenburg-Strelitz):

In [ ]:
louise_img = np.float32(PIL.Image.open('images/louise.jpg'))
showarray(louise_img)

Running the next code cell starts the detail generation process. You may see how new patterns start to form, iteration by iteration, octave by octave.

In [ ]:
emotion_stages=['conv5', 'conv3', 'conv4', 'conv4', 'conv4']
    
dream = emotions.long_dream(louise_img, stages=emotion_stages, resize_in=(224, 224), resize_out=(800, 800))
showarray(dream)

In [ ]:
emotions.net.blobs.keys()

## Controlling dreams

The image detail generation method described above tends to produce some patterns more often the others. One easy way to improve the generated image diversity is to tweak the optimization objective. Here we show just one of many ways to do that. Let's use one more input image. We'd call it a "*guide*".

In [ ]:
guide = np.float32(PIL.Image.open('images/pattern_city.jpg'))
guide = resizearray(guide, 224, 224)
showarray(guide)

Now we pick some target layer and extract guide image features.

In [ ]:
obsession_end = 'conv4'
dst, guide_features = define_obsession(emotions.net, end=obsession_end, guide=guide)
objective_guide = make_objective_guide(guide_features)

_=emotions.deepdream(louise_img, end=obsession_end, objective=objective_guide)